In [ ]:
'''
Clayton Cohn
20200217
CSC440
Assignment 4
'''

'''
1. (10 points) Exercise 4, page 147

    For a string of bits S, let Sbar denote the complementary string obtained by changing all the 1s to 0s
    and all the 0s to 1s (equivalently, Sbar = S ^ 11111....). Show that if DES key K encrypts P to C, then
    Kbar encrypts Pbar to Cbar. (Hint: This has nothing to do with the structure of the S-boxes. To do the
    problem, just work through the encryption algorithm).
'''

In [ ]:
'''
2. (10 points) Exercise 6 on page 147

    Suppose Triple DES is performed by choosing two keys K1, K2 and computing Ek1(Ek2(Ek2(m))) (note that the order
    of the keys has been modified from the usual two-key version of Triple DES). Show how to attack this modified 
    version with a meet-in-the-middle attack.
'''

In [ ]:
'''
3. (50 points) Implement encryption in the textbook's simplified DES scheme. 

    Write the program so that it contains a method called encrypt(plaintext, key) 
    where the plaintext and the key are both integers. The first must be in the range 0 to 4095 
    and the second in the range 0 to 511. My suggestion is to write this method with the first part 
    generating a list or array of the four sub-keys and the second part a loop that iterates four times, 
    once for each round. In that loop, call methods for carrying out an exclusive-or, for expanding, 
    and for applying each of the two S-boxes. I am leaving a number of details unspecified but those will 
    be filled in as you begin to work on this and ask me questions. I will provide test data in the form 
    of plaintexts, keys, and their resulting ciphertexts.

    Please also add to the program a driver method for testing your encryption. This method should prompt 
    for the plaintext and the key as integers, call the encryption method, and then output the ciphertext.


'''